In [1]:
import uproot4
import awkward1 as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep
import numba as nb
import h5py

In [ ]:
import os
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000")
#os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000")
os.listdir("/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001")

In [ ]:
root_ = uproot4.open( "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root" )
print ( "Number of events in tree: {}".format( np.array( root_["demo/SlimmedNtuple/event"] ).size ) )

tree_ = root_["demo/SlimmedNtuple"]
 
keys_nonproton = ["run", "event", "lumiblock",
        "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
        "pfcand_nextracks", "pfcand_nextracks_noDRl",
        "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi"]
keys_nonproton.extend( tree_.keys( filter_name="jet*") )
keys_nonproton.extend( tree_.keys( filter_name="muon*") )
keys = []
keys.extend( keys_nonproton )
keys.extend( tree_.keys( filter_name="proton*") )
keys.extend( tree_.keys( filter_name="pps*") )
print ( keys )

In [ ]:
arrays = tree_.arrays( keys, library="ak", how="zip", entry_stop=10 )
#arrays[ "slice" ] = np.zeros( len(arrays), dtype=np.int32 )
ak.to_list( arrays )
#ak.to_list( arrays["pps_track"] )

In [ ]:
protons = arrays["proton"]
protons["run"] = arrays["run"]
protons["event"] = arrays["event"]
protons["lumiblock"] = arrays["lumiblock"]
ppstracks = arrays["pps_track"]
ppstracks["run"] = arrays["run"]
ppstracks["event"] = arrays["event"]
ppstracks["lumiblock"] = arrays["lumiblock"]
print ( ak.to_list(protons) )
print ("\n")
print ( ak.to_list(ppstracks) )

In [2]:
debug = False
fileNames = [
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017D-Dilepton/190621_214154/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_1.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_0.root",
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0000/SlimmedNtuple_merged_1.root"
    #"/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017F-Dilepton/190621_214253/0001/SlimmedNtuple_merged_0.root"
]
firstEvent_ = 0
entrystop_ = None
#label = "test"
#label = "data"
label = "data-random-resample_20"
read_size_ = "150MB"
#random_protons = False
#resample_factor = -1
random_protons = True
resample_factor = 20

resample = False
if resample_factor > 1: resample = True
print ( "Resample: {} / Resample factor: {}".format( resample,  resample_factor ) )

np.random.seed( 42 )

dset_chunk_size = 50000

columns_protons = ( "run", "lumiblock", "event", "slice", "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm",
                    "jet0_pt", "jet0_eta", "jet0_phi", "jet0_energy", "jet0_mass", "jet0_corrmass", "jet0_tau1", "jet0_tau2", "jet0_vertexz",
                    "muon0_pt", "muon0_eta", "muon0_phi", "muon0_energy", "muon0_charge", "muon0_iso", "muon0_dxy", "muon0_dz",
                    "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
                    "pfcand_nextracks", "pfcand_nextracks_noDRl",
                    "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" )

columns_ppstracks = ( "run", "lumiblock", "event", "slice", "x", "y", "rpid" ) 

protons_keys = {}
for col_ in columns_protons:
    protons_keys[ col_ ] = col_

ppstracks_keys = {}
for col_ in columns_ppstracks:
    ppstracks_keys[ col_ ] = col_

counts_label_protons_ = "Proton" if not random_protons else "ProtonRnd"

with h5py.File( 'output-' + label + '.h5', 'w') as f:

    dset_protons_multiRP = f.create_dataset( 'protons_multiRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
    print ( "Initial dataset shape: {}".format( dset_protons_multiRP.shape ) )

    dset_protons_singleRP = f.create_dataset( 'protons_singleRP', ( dset_chunk_size, len( columns_protons ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_protons ) ) )
    "/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_1.root",
    
    print ( "Initial dataset shape: {}".format( dset_protons_singleRP.shape ) )
    
    dset_ppstracks = f.create_dataset( 'ppstracks', ( dset_chunk_size, len( columns_ppstracks ) ), compression="gzip", chunks=True, maxshape=( None , len( columns_ppstracks ) ) )
    print ( "Initial dataset shape: {}".format( dset_ppstracks.shape ) )
    
    protons_multiRP_list = {}
    for col_ in columns_protons:
        protons_multiRP_list[ col_ ] = []           

    protons_singleRP_list = {}
    for col_ in columns_protons:
        protons_singleRP_list[ col_ ] = []           

    ppstracks_list = {}
    for col_ in columns_ppstracks:
        ppstracks_list[ col_ ] = []           

    selections = None
    counts = None
    
    dset_multiRP_slice = 0
    dset_multiRP_idx = 0
    dset_multiRP_entries = 0

    dset_singleRP_slice = 0
    dset_singleRP_idx = 0
    dset_singleRP_entries = 0
    
    dset_ppstracks_slice = 0
    dset_ppstracks_idx = 0
    dset_ppstracks_entries = 0
    
    for file_ in fileNames:
        print ( file_ ) 
        root_ = uproot4.open( file_ )

        print ( "Number of events in tree: {}".format( np.array( root_["demo/SlimmedNtuple/event"] ).size ) )

        tree_ = root_["demo/SlimmedNtuple"]

        keys_nonproton = [ "run", "event", "lumiblock",
                           "num_bjets_ak8", "num_bjets_ak4", "num_jets_ak4",
                           "pfcand_nextracks", "pfcand_nextracks_noDRl",
                           "recoMWhad", "recoMWlep", "recoMWW", "recoRapidityWW", "dphiWW", "WLeptonicPt", "WLeptonicPhi" ]
        keys_nonproton.extend( tree_.keys( filter_name="jet*") )
        keys_nonproton.extend( tree_.keys( filter_name="muon*") )
        keys = []
        keys.extend( keys_nonproton )
        keys.extend( tree_.keys( filter_name="proton*") )
        keys.extend( tree_.keys( filter_name="pps*") )
        print ( keys )

        for events_ in tree_.iterate( keys , library="ak", how="zip", step_size=read_size_, entry_start=firstEvent_, entry_stop=entrystop_ ):
            print ( len(events_), events_ )
            print ( "Num jets: {}".format( ak.num( events_["jet"] ) ) )
            print ( "Num muons: {}".format( ak.num( events_["muon"] ) ) )
            print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
            print ( "Num pps tracks: {}".format( ak.num( events_["pps_track"] ) ) )

            selections_ = []
            counts_ = []
            
            selections_.append( "All" )
            counts_.append( len( events_ ) )
            
            # Event selections
            msk_1jet = ( ak.num( events_["jet"] ) >= 1 )
            selections_.append( "Jet" )
            counts_.append( np.sum( np.array( msk_1jet ).astype("int32") ) )
            
            msk_1muon = msk_1jet & ( ak.num( events_["muon"] ) >= 1 )
            selections_.append( "Muon" )
            counts_.append( np.sum( np.array( msk_1muon ).astype("int32") ) )
            
            events_ = events_[ msk_1muon ]    
                             
            selections_ = np.array( selections_ )
            counts_ = np.array( counts_ )
                             
            # Repeat events by resample factor
            if resample:
                counts_ = counts_ * resample_factor
                
            if selections is None:
                selections = selections_
                counts = counts_
            else:
                msk_selections = np.full_like( selections, False, dtype='bool' )
                for key in selections_:
                    msk_selections |= ( selections == key )
                counts[ msk_selections ] += counts_
            
            # Repeat events by resample factor
            slices_ = np.zeros( len( events_ ), dtype=np.int32 )
            if resample:
                events_size_ = len( events_ )
                events_ = ak.concatenate( ( [events_] * resample_factor ), axis=0 )
                slices_ = np.zeros( resample_factor * events_size_, dtype=np.int32 )
                for idx_ in range( resample_factor ):
                    slices_[ ( idx_ * events_size_ ) : ( ( idx_ + 1 ) * events_size_ ) ] = idx_
            
            events_[ "slice" ] = slices_
                
            # Randomize proton arrays
            #protons_ = events_["proton"]
            protons_ = None
            if random_protons:
                protons_ = events_["proton"]
            
                index_rnd_ = np.random.permutation( len( events_ ) )
            
                protons_rnd_ = protons_[ index_rnd_ ]
            
                events_[ "proton_rnd" ] = protons_rnd_    
        
                print ( "Num protons: {}".format( ak.num( events_["proton"] ) ) )
                print ( "Num protons randomized: {}".format( ak.num( events_["proton_rnd"] ) ) )    
            
            if not random_protons: protons_ = events_["proton"]
            else:                  protons_ = events_["proton_rnd"]
            print ( "Num protons: {}".format( ak.num( protons_ ) ) )
            
            protons_["run"]                    = events_["run"]
            protons_["lumiblock"]              = events_["lumiblock"]
            protons_["event"]                  = events_["event"]
            protons_["slice"]                  = events_["slice"]
            protons_["jet0_pt"]                = events_.jet.pt[:,0]
            protons_["jet0_eta"]               = events_.jet.eta[:,0]
            protons_["jet0_phi"]               = events_.jet.phi[:,0]
            protons_["jet0_energy"]            = events_.jet.energy[:,0]
            protons_["jet0_mass"]              = events_.jet.mass[:,0]
            protons_["jet0_corrmass"]          = events_.jet.corrmass[:,0]
            protons_["jet0_tau1"]              = events_.jet.tau1[:,0]
            protons_["jet0_tau2"]              = events_.jet.tau2[:,0]
            protons_["jet0_vertexz"]           = events_.jet.vertexz[:,0]
            protons_["muon0_pt"]               = events_.muon.pt[:,0]
            protons_["muon0_eta"]              = events_.muon.eta[:,0]
            protons_["muon0_phi"]              = events_.muon.phi[:,0]
            protons_["muon0_energy"]           = events_.muon.e[:,0]
            protons_["muon0_charge"]           = events_.muon.charge[:,0]
            protons_["muon0_iso"]              = events_.muon.iso[:,0]
            protons_["muon0_dxy"]              = events_.muon.dxy[:,0]
            protons_["muon0_dz"]               = events_.muon.dz[:,0]
            protons_["num_bjets_ak8"]          = events_["num_bjets_ak8"]
            protons_["num_bjets_ak4"]          = events_["num_bjets_ak4"]
            protons_["num_jets_ak4"]           = events_["num_jets_ak4"]
            protons_["pfcand_nextracks"]       = events_["pfcand_nextracks"]
            protons_["pfcand_nextracks_noDRl"] = events_["pfcand_nextracks_noDRl"]
            protons_["recoMWhad"]              = events_["recoMWhad"]
            protons_["recoMWlep"]              = events_["recoMWlep"]
            protons_["recoMWW"]                = events_["recoMWW"]
            protons_["recoRapidityWW"]         = events_["recoRapidityWW"]
            protons_["dphiWW"]                 = events_["dphiWW"]
            protons_["WLeptonicPt"]            = events_["WLeptonicPt"]
            protons_["WLeptonicPhi"]           = events_["WLeptonicPhi"]
            #protons_["x1"] = -999.
            #protons_["y1"] = -999.
            #protons_["x2"] = -999.
            #protons_["y2"] = -999.
            ppstracks_ = events_["pps_track"]
            ppstracks_["run"] = events_["run"]
            ppstracks_["lumiblock"] = events_["lumiblock"]
            ppstracks_["event"] = events_["event"]
            ppstracks_["slice"] = events_["slice"]

            protons_singleRP_ = protons_[ protons_.ismultirp == 0 ]
            protons_multiRP_ = protons_[ protons_.ismultirp == 1 ]

            protons_singleRP_byRP_ = {}
            ppstracks_byRP_ = {}
            protons_multiRP_byArm_ = {}
            for rpid in ( 3, 23, 103, 123 ):
                #arm = -1
                #if   rpid == 3   or rpid == 23 : arm = 0
                #elif rpid == 103 or rpid == 123 : arm = 1
                #print ( "Arm: {}".format( arm ) )

                protons_singleRP_byRP_[ rpid ] =  protons_singleRP_[ protons_singleRP_.rpid == rpid ]
                ppstracks_byRP_[ rpid ] = ppstracks_[ ppstracks_.rpid == rpid ]
                #protons_singleRP_byRP_[ rpid ]["x1"] = ppstracks_byRP_[ rpid ].x
                #protons_singleRP_byRP_[ rpid ]["y1"] = ppstracks_byRP_[ rpid ].y

                print ( "\nNum protons RP {}: {}".format( rpid, ak.num( protons_singleRP_byRP_[ rpid ] ) ) )
                if debug:
                    print ( ak.to_list( protons_singleRP_byRP_[ rpid ] ) )
                    print ("\n")
                    print ( ak.to_list( ppstracks_byRP_[ rpid ] ) )

            for arm in ( 0, 1 ):
                protons_multiRP_byArm_[ arm ] = protons_multiRP_[ protons_multiRP_.arm == arm ]

                print ( "\nNum multi-RP protons Arm {}: {}".format( arm, ak.num( protons_multiRP_byArm_[ arm ] ) ) )
                if debug:
                    print ( ak.to_list( protons_multiRP_byArm_[ arm ] ) )

            #msk  =  np.array( ak.num( protons_singleRP_byRP_[ 3 ].xi ) == 1 )
            #msk &= np.array( ak.num( protons_singleRP_byRP_[ 23 ].xi ) == 1 )
            #msk &= np.array( ak.num( protons_singleRP_byRP_[ 103 ].xi ) == 1 )
            #msk &= np.array( ak.num( protons_singleRP_byRP_[ 123 ].xi ) == 1 )    

            msk_protons  = np.array( ak.num( protons_multiRP_byArm_[ 0 ] ) > 0 )
            msk_protons &= np.array( ak.num( protons_multiRP_byArm_[ 1 ] ) > 0 )

            protons_multiRP_sel_ = protons_multiRP_[ msk_protons ]
            protons_singleRP_sel_ = protons_singleRP_[ msk_protons ]
            ppstracks_sel_ = ppstracks_[ msk_protons ]
            print ("\n")
            if debug:
                print ( msk_protons )
            print ( len( protons_multiRP_sel_ ) )
            print ( ak.num( protons_multiRP_sel_ ) )
            if debug:
                print ("\n")
                print ( ak.to_list( protons_multiRP_sel_ ) )
                print ("\n")
                print ( ak.to_list( protons_singleRP_sel_ ) )
                print ("\n")
                print ( ak.to_list( ppstracks_sel_ ) )

            counts_protons_ = len( protons_[ msk_protons ] )
            if not counts_label_protons_ in selections:
                selections = np.concatenate( ( selections, np.array( [ counts_label_protons_ ] ) ) )
                counts = np.concatenate( ( counts, np.array( [counts_protons_] ) ) )
            else:    
                counts[ selections == counts_label_protons_ ] += counts_protons_ 
    
            print ( selections )
            print ( counts )
                             
            for col_ in columns_protons:
                protons_multiRP_list[ col_ ] = np.array( ak.flatten( protons_multiRP_sel_[ protons_keys[ col_ ] ] ) )

            arr_size_multiRP_ = len( protons_multiRP_list[ "xi" ] )
            print ( "Flattened array size multi-RP: {}".format( arr_size_multiRP_ ) )

            for col_ in columns_protons:
                protons_singleRP_list[ col_ ] = np.array( ak.flatten( protons_singleRP_sel_[ protons_keys[ col_ ] ] ) )

            arr_size_singleRP_ = len( protons_singleRP_list[ "xi" ] )
            print ( "Flattened array size single-RP: {}".format( arr_size_singleRP_ ) )

            for col_ in columns_ppstracks:
                ppstracks_list[ col_ ] = np.array( ak.flatten( ppstracks_sel_[ ppstracks_keys[ col_ ] ] ) )

            arr_size_ppstracks_ = len( ppstracks_list[ "x" ] )
            print ( "Flattened array size tracks: {}".format( arr_size_ppstracks_ ) )

            dset_multiRP_entries += arr_size_multiRP_
            dset_singleRP_entries += arr_size_singleRP_
            dset_ppstracks_entries += arr_size_ppstracks_

            if dset_multiRP_entries > dset_chunk_size:
                resize_factor_ = ( dset_multiRP_entries // dset_chunk_size )
                chunk_resize_  = resize_factor_ * dset_chunk_size

                print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
                dset_protons_multiRP.resize( ( dset_protons_multiRP.shape[0] + chunk_resize_ ), axis=0 )
                print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )
                        
                dset_multiRP_slice += resize_factor_
                # Count the rest to the chunk size 
                dset_multiRP_entries = ( dset_multiRP_entries % dset_chunk_size )
                             
            if dset_singleRP_entries > dset_chunk_size:
                resize_factor_ = ( dset_singleRP_entries // dset_chunk_size )
                chunk_resize_  = resize_factor_ * dset_chunk_size

                print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
                dset_protons_singleRP.resize( ( dset_protons_singleRP.shape[0] + chunk_resize_ ), axis=0 )
                print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )
                        
                dset_singleRP_slice += resize_factor_
                # Count the rest to the chunk size 
                dset_singleRP_entries = ( dset_singleRP_entries % dset_chunk_size )
                             
            if dset_ppstracks_entries > dset_chunk_size:
                resize_factor_ = ( dset_ppstracks_entries // dset_chunk_size )
                chunk_resize_  = resize_factor_ * dset_chunk_size

                print ( "Resizing output dataset by {} entries.".format( chunk_resize_ ) )
                dset_ppstracks.resize( ( dset_ppstracks.shape[0] + chunk_resize_ ), axis=0 )
                print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )
                        
                dset_ppstracks_slice += resize_factor_
                # Count the rest to the chunk size 
                dset_ppstracks_entries = ( dset_ppstracks_entries % dset_chunk_size )
                             
            print ( "Stacking data." )
            data_protons_multiRP_ = np.stack( list( protons_multiRP_list.values() ), axis=1 )
            print ( data_protons_multiRP_.shape )
            print ( data_protons_multiRP_ )

            data_protons_singleRP_ = np.stack( list( protons_singleRP_list.values() ), axis=1 )
            print ( data_protons_singleRP_.shape )
            print ( data_protons_singleRP_ )

            data_ppstracks_ = np.stack( list( ppstracks_list.values() ), axis=1 )
            print ( data_ppstracks_.shape )
            print ( data_ppstracks_ )

            dset_idx_next_ = dset_multiRP_idx + arr_size_multiRP_
            print ( "Slice: {}".format( dset_multiRP_slice ) )
            print ( "Writing in positions ({},{})".format( dset_multiRP_idx, dset_idx_next_ ) )
            dset_protons_multiRP[ dset_multiRP_idx : dset_idx_next_ ] = data_protons_multiRP_
            dset_multiRP_idx = dset_idx_next_ 

            dset_idx_next_ = dset_singleRP_idx + arr_size_singleRP_
            print ( "Slice: {}".format( dset_singleRP_slice ) )
            print ( "Writing in positions ({},{})".format( dset_singleRP_idx, dset_idx_next_ ) )
            dset_protons_singleRP[ dset_singleRP_idx : dset_idx_next_ ] = data_protons_singleRP_
            dset_singleRP_idx = dset_idx_next_ 

            dset_idx_next_ = dset_ppstracks_idx + arr_size_ppstracks_
            print ( "Slice: {}".format( dset_ppstracks_slice ) )
            print ( "Writing in positions ({},{})".format( dset_ppstracks_idx, dset_idx_next_ ) )
            dset_ppstracks[ dset_ppstracks_idx : dset_idx_next_ ] = data_ppstracks_
            dset_ppstracks_idx = dset_idx_next_ 

        # Iteration on input files
        root_.close()

    # Reduce dataset to its final size 
    print ( "Reduce dataset." )
    dset_protons_multiRP.resize( ( dset_multiRP_idx ), axis=0 ) 
    print ( "Dataset shape: {}".format( dset_protons_multiRP.shape ) )

    dset_protons_singleRP.resize( ( dset_singleRP_idx ), axis=0 ) 
    print ( "Dataset shape: {}".format( dset_protons_singleRP.shape ) )

    dset_ppstracks.resize( ( dset_ppstracks_idx ), axis=0 ) 
    print ( "Dataset shape: {}".format( dset_ppstracks.shape ) )

    print ( "Writing column names and event counts.")

    columns_protons_ = np.array( columns_protons, dtype='S' )
    print ( columns_protons_ )

    columns_ppstracks_ = np.array( columns_ppstracks, dtype='S' )
    print ( columns_ppstracks_ )

    event_counts_ = counts
    print ( event_counts_ )

    selections_ = np.array( selections, dtype='S' )
    print ( selections_ )

    dset_columns_protons = f.create_dataset( 'columns_protons', data=columns_protons_ )
    dset_columns_ppstracks = f.create_dataset( 'columns_ppstracks', data=columns_ppstracks_ )
    dset_counts = f.create_dataset( 'event_counts', data=event_counts_ )
    dset_selections = f.create_dataset( 'selections', data=selections_ )

    print ( dset_protons_multiRP )
    print ( dset_protons_multiRP[-1] )
    print ( dset_protons_singleRP )
    print ( dset_protons_singleRP[-1] )   
    print ( dset_ppstracks )
    print ( dset_ppstracks[-1] )   

    print ( dset_columns_protons )
    print ( list( dset_columns_protons ) )
    print ( dset_columns_ppstracks )
    print ( list( dset_columns_ppstracks ) )   
    print ( dset_counts )
    print ( list( dset_counts ) )
    print ( dset_selections )
    print ( list( dset_selections ) )


Resample: True / Resample factor: 20
Initial dataset shape: (50000, 40)
Initial dataset shape: (50000, 40)
Initial dataset shape: (50000, 7)
/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017B-Dilepton/190621_214052/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 5153
['run', 'event', 'lumiblock', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'pps_track_x', 'pps_track_y', 'pps_track_rpi

/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017C-Dilepton/190621_214124/0000/SlimmedNtuple_merged_1.root
Number of events in tree: 6936
['run', 'event', 'lumiblock', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'pps_track_x', 'pps_track_y', 'pps_track_rpid']
6936 [{run: 300811, event: 292345398, lumiblock: 170, num_bjets_ak8: 0, ... rpid: 23}]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

/eos/user/a/antoniov/Workspace/analysis/data/PPS/SingleMuon-Run2017E-Dilepton/190621_214224/0000/SlimmedNtuple_merged_0.root
Number of events in tree: 11693
['run', 'event', 'lumiblock', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'pps_track_x', 'pps_track_y', 'pps_track_rpid']
11693 [{run: 304158, event: 1258131585, lumiblock: 981, ... pps_track: []}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 

Number of events in tree: 12900
['run', 'event', 'lumiblock', 'num_bjets_ak8', 'num_bjets_ak4', 'num_jets_ak4', 'pfcand_nextracks', 'pfcand_nextracks_noDRl', 'recoMWhad', 'recoMWlep', 'recoMWW', 'recoRapidityWW', 'dphiWW', 'WLeptonicPt', 'WLeptonicPhi', 'jet_pt', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_phi', 'jet_eta', 'jet_mass', 'jet_tau1', 'jet_tau2', 'jet_corrmass', 'jet_vertexz', 'jet_jer_res', 'jet_jer_sf', 'jet_jer_sfup', 'jet_jer_sfdown', 'muon_pt', 'muon_eta', 'muon_phi', 'muon_px', 'muon_py', 'muon_pz', 'muon_e', 'muon_charge', 'muon_iso', 'muon_dxy', 'muon_dz', 'proton_xi', 'proton_thx', 'proton_thy', 'proton_t', 'proton_ismultirp_', 'proton_rpid', 'proton_arm', 'pps_track_x', 'pps_track_y', 'pps_track_rpid']
12900 [{run: 305351, event: 442789068, lumiblock: 248, num_bjets_ak8: 0, ... rpid: 123}]}]
Num jets: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Num muons: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ... 2, 2, 2, 2, 2, 2

Slice: 3
Writing in positions (172626,197660)
Reduce dataset.
Dataset shape: (86520, 40)
Dataset shape: (173040, 40)
Dataset shape: (197660, 7)
Writing column names and event counts.
[b'run' b'lumiblock' b'event' b'slice' b'xi' b'thx' b'thy' b't'
 b'ismultirp' b'rpid' b'arm' b'jet0_pt' b'jet0_eta' b'jet0_phi'
 b'jet0_energy' b'jet0_mass' b'jet0_corrmass' b'jet0_tau1' b'jet0_tau2'
 b'jet0_vertexz' b'muon0_pt' b'muon0_eta' b'muon0_phi' b'muon0_energy'
 b'muon0_charge' b'muon0_iso' b'muon0_dxy' b'muon0_dz' b'num_bjets_ak8'
 b'num_bjets_ak4' b'num_jets_ak4' b'pfcand_nextracks'
 b'pfcand_nextracks_noDRl' b'recoMWhad' b'recoMWlep' b'recoMWW'
 b'recoRapidityWW' b'dphiWW' b'WLeptonicPt' b'WLeptonicPhi']
[b'run' b'lumiblock' b'event' b'slice' b'x' b'y' b'rpid']
[1570080 1570080 1560700   43260]
[b'All' b'Jet' b'Muon' b'ProtonRnd']
<HDF5 dataset "protons_multiRP": shape (86520, 40), type "<f4">
[ 3.0609200e+05  3.3200000e+02  6.2494829e+08  1.9000000e+01
  1.2374489e-01 -1.2627274e-05 -2.9034938

In [ ]:
df_protons_multiRP_list = []
df_protons_singleRP_list = []
df_ppstracks_list = []
df_counts_list = []

#resample_factor = 20

fileNames = [
    'output-test.h5'
]

for file_ in fileNames:
    print ( file_ )
    with h5py.File( file_, 'r' ) as f:
        print ( list(f.keys()) )
        
        dset_protons_multiRP = f['protons_multiRP']
        print ( dset_protons_multiRP.shape )
        print ( dset_protons_multiRP[:,:] )
        
        dset_protons_singleRP = f['protons_singleRP']
        print ( dset_protons_singleRP.shape )
        print ( dset_protons_singleRP[:,:] )

        dset_ppstracks = f['ppstracks']
        print ( dset_ppstracks.shape )
        print ( dset_ppstracks[:,:] )

        dset_columns_protons = f['columns_protons']
        print ( dset_columns_protons.shape )
        columns_protons = list( dset_columns_protons )
        print ( columns_protons )
        columns_protons_str = [ item.decode("utf-8") for item in columns_protons ]
        print ( columns_protons_str )

        dset_columns_ppstracks = f['columns_ppstracks']
        print ( dset_columns_ppstracks.shape )
        columns_ppstracks = list( dset_columns_ppstracks )
        print ( columns_ppstracks )
        columns_ppstracks_str = [ item.decode("utf-8") for item in columns_ppstracks ]
        print ( columns_ppstracks_str )

        dset_selections = f['selections']
        selections_ = [ item.decode("utf-8") for item in dset_selections ]
        print ( selections_ )
        
        dset_counts = f['event_counts']
        df_counts_list.append( pd.Series( dset_counts, index=selections_ ) )
        print ( df_counts_list[-1] )
       
        chunk_size = 1000000
        entries_protons_multiRP = dset_protons_multiRP.shape[0]
        start_ = list( range( 0, entries_protons_multiRP, chunk_size ) )
        stop_ = start_[1:]
        stop_.append( entries_protons_multiRP )
        print ( start_ )
        print ( stop_ )
        for idx in range( len( start_ ) ):
            print ( start_[idx], stop_[idx] )
            #print ( dset[ start_[idx] : stop_[idx] ] )
            df_ = pd.DataFrame( dset_protons_multiRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
                                    { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
                                      "muon0_charge": "int32",
                                      "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
                                      "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
            df_protons_multiRP_list.append( df_ )
            print ( df_protons_multiRP_list[-1].head() )
            print ( "Data set size: {}".format( len( df_protons_multiRP_list[-1] ) ) )

        entries_protons_singleRP = dset_protons_singleRP.shape[0]
        start_ = list( range( 0, entries_protons_singleRP, chunk_size ) )
        stop_ = start_[1:]
        stop_.append( entries_protons_singleRP )
        print ( start_ )
        print ( stop_ )
        for idx in range( len( start_ ) ):
            print ( start_[idx], stop_[idx] )
            #print ( dset[ start_[idx] : stop_[idx] ] )
            df_ = pd.DataFrame( dset_protons_singleRP[ start_[idx] : stop_[idx] ], columns=columns_protons_str ).astype(
                                    { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "ismultirp": "int32", "rpid": "int32", "arm": "int32",
                                      "muon0_charge": "int32",
                                      "num_bjets_ak8": "int32", "num_bjets_ak4": "int32", "num_jets_ak4": "int32",
                                      "pfcand_nextracks": "int32", "pfcand_nextracks_noDRl": "int32" } )
            df_protons_singleRP_list.append( df_ )
            print ( df_protons_singleRP_list[-1].head() )
            print ( "Data set size: {}".format( len( df_protons_singleRP_list[-1] ) ) )

        entries_ppstracks = dset_ppstracks.shape[0]
        start_ = list( range( 0, entries_ppstracks, chunk_size ) )
        stop_ = start_[1:]
        stop_.append( entries_ppstracks )
        print ( start_ )
        print ( stop_ )
        for idx in range( len( start_ ) ):
            print ( start_[idx], stop_[idx] )
            #print ( dset[ start_[idx] : stop_[idx] ] )
            df_ = pd.DataFrame( dset_ppstracks[ start_[idx] : stop_[idx] ], columns=columns_ppstracks_str ).astype( { "run": "int64", "lumiblock": "int64", "event": "int64", "slice": "int32", "rpid": "int32" } )
            df_ppstracks_list.append( df_ )
            print ( df_ppstracks_list[-1].head() )
            print ( "Data set size: {}".format( len( df_ppstracks_list[-1] ) ) )

df_counts = df_counts_list[0]
for idx in range( 1, len( df_counts_list ) ):
    df_counts = df_counts.add( df_counts_list[idx] )
print ( df_counts )

df_protons_multiRP = pd.concat( df_protons_multiRP_list )
print (df_protons_multiRP)

df_protons_singleRP = pd.concat( df_protons_singleRP_list )
print (df_protons_singleRP)

df_ppstracks = pd.concat( df_ppstracks_list )
print (df_ppstracks)


In [ ]:
df_protons_multiRP[ ['run', 'lumiblock', 'event', 'slice'] ]

In [ ]:
df_protons_multiRP_index = df_protons_multiRP.set_index( ['run', 'lumiblock', 'event', 'slice'] )
df_protons_multiRP_index

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP[ df_protons_multiRP.arm == 0 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[0] )
df_protons_multiRP[ df_protons_multiRP.arm == 1 ].hist( "xi", bins=10, range=(0.,0.2), ax=axes[1] )

In [ ]:
df_protons_multiRP_events = df_protons_multiRP_index.drop( columns=[ "xi", "thx", "thy", "t", "ismultirp", "rpid", "arm" ] )
df_protons_multiRP_events = df_protons_multiRP_events[ ~df_protons_multiRP_events.index.duplicated(keep='first') ]
df_protons_multiRP_events

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "recoMWW", bins=50, range=(200.,2000.), ax=axes[0] )
df_protons_multiRP_events.hist( "recoRapidityWW", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
#list( df_protons_multiRP_index.groupby( ["run","lumiblock","event"] ) )
#df_protons_multiRP_index.groupby( ["run","lumiblock","event"] ).apply( lambda df_: print( df_.iloc[0].xi, df_.iloc[1].xi ) ) 
df_protons_multiRP_events[ "MX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event"] ).apply( lambda df_: 13000. * np.sqrt( df_.iloc[0].xi * df_.iloc[1].xi ) )
df_protons_multiRP_events[ "YX" ] = df_protons_multiRP_index.groupby( ["run","lumiblock","event"] ).apply( lambda df_: 0.5 * np.log( df_.iloc[0].xi / df_.iloc[1].xi ) )
df_protons_multiRP_events[ "diffMWW_MX" ]  = df_protons_multiRP_events[ "recoMWW" ] - df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "ratioMWW_MX" ] = df_protons_multiRP_events[ "recoMWW" ] / df_protons_multiRP_events[ "MX" ]
df_protons_multiRP_events[ "diffYWW_YX" ]  = df_protons_multiRP_events[ "recoRapidityWW" ] - df_protons_multiRP_events[ "YX" ]
df_protons_multiRP_events

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "MX", bins=50, range=(200.,2000.), ax=axes[0] )
df_protons_multiRP_events.hist( "YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_protons_multiRP_events.hist( "diffMWW_MX", bins=50, range=(-1000.,1000.), ax=axes[0] )
df_protons_multiRP_events.hist( "diffYWW_YX", bins=50, range=(-3.,3.), ax=axes[1] )

In [ ]:
df_protons_multiRP_events.hist( "ratioMWW_MX", bins=50, range=(0.,1.5), figsize=(12,10) )

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.plot( (-2000.,2000.), (0.,0.), color='lightblue' )
plt.plot( (0.,0.), (-3.,3.), color='lightblue' )
ax = plt.gca()
df_protons_multiRP_events.plot( "diffMWW_MX", "diffYWW_YX", 'scatter', ax=ax )
plt.xlim(-2000., 2000.)
plt.ylim(-3., 3.)

In [ ]:
df_protons_singleRP

In [ ]:
df_protons_singleiRP_index = df_protons_singleRP.set_index( ['run', 'lumiblock', 'event'] )
df_protons_singleiRP_index

In [ ]:
df_ppstracks

In [ ]:
df_ppstracks_index = df_ppstracks.set_index( ['run', 'lumiblock', 'event'] )
#df_ppstracks_index
df_ppstracks_index[ ( df_ppstracks_index.rpid != 16 ) & ( df_ppstracks_index.rpid != 116 ) ]

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(24,10) )
df_ppstracks[ df_ppstracks.rpid == 103 ].hist( "x", bins=10, range=(0.,20.), ax=axes[0] )
df_ppstracks[ df_ppstracks.rpid == 103 ].hist( "y", bins=10, range=(-15.,15.), ax=axes[1] )

### References

In [ ]:
np.info( ak.concatenate )

In [ ]:
np.info( pd.DataFrame.hist )

In [ ]:
np.info( pd.DataFrame.plot.scatter )

In [ ]:
np.info( plt.plot )